# **<font color='crimson'>Finance</font>**

---

In [ ]:
###

In [ ]:
SELECT year, month, revenue,
    LAG(revenue, 1) OVER w AS prev,
    ROUND(revenue * 100.0 / lag(revenue, 1) OVER w) AS perc
FROM sales
WHERE year = 2020 AND plan = 'gold'
WINDOW w AS (
    ORDER BY month)
ORDER BY month;

In [ ]:
###

In [ ]:
SELECT plan, year, month, revenue,
    SUM(revenue) OVER w AS total
FROM sales
WHERE quarter = 1 AND year = 2020
WINDOW w AS (
    PARTITION BY plan
    ORDER BY month)
ORDER BY plan, month;

In [ ]:
###

In [ ]:
SELECT
    year,
    month,
    revenue,
    ROUND(AVG(revenue) OVER w, 0) AS avg3m
FROM sales
WHERE year = 2020 AND plan = 'platinum'
WINDOW w AS (
    ORDER BY month
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
ORDER BY month;

In [ ]:
###

In [ ]:
SELECT
    year,
    month,
    revenue,
    last_value(revenue) OVER w AS december,
    ROUND(
        revenue * 100.0 / last_value(revenue) OVER w, 0) AS perc
FROM sales
WHERE plan = 'silver'
WINDOW w AS (
    PARTITION BY year
    ORDER BY month
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
ORDER BY year, month

In [ ]:
###

In [ ]:
WITH data AS (
SELECT
    year,
    plan,
    sum(revenue) AS revenue
FROM sales
GROUP BY year, plan
ORDER BY year, plan)

SELECT
    year,
    plan,
    revenue,
    SUM(revenue) OVER w AS total,
    ROUND(
        revenue * 100.0 / SUM(revenue) OVER w, 0) AS perc
FROM data
WINDOW w AS (
    PARTITION BY year
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
ORDER BY year, plan;

In [ ]:
###

In [ ]:
WITH data AS (SELECT
    year,
    month,
    SUM(revenue) AS revenue
FROM sales
WHERE year = 2020
GROUP BY year, month
ORDER BY revenue DESC)

SELECT
    year,
    month,
    revenue,
    ntile(3) OVER w AS tile
FROM data
WINDOW w AS (
    ORDER BY revenue DESC)
ORDER BY revenue DESC, month;

In [ ]:
###

In [ ]:
WITH data AS(
SELECT
    year,
    quarter,
    SUM(CASE WHEN year = 2020 THEN revenue END) AS revenue,
    LAG(SUM(CASE WHEN year = 2019 THEN revenue END)) OVER w AS prev
FROM sales
GROUP BY year, quarter
WINDOW w AS (
    PARTITION BY quarter
    ORDER BY year, quarter)
ORDER BY year, quarter)

SELECT
    year,
    quarter,
    revenue,
    prev,
    ROUND(revenue * 100.0 / prev, 0) AS perc
FROM data
WHERE year = 2020
ORDER BY quarter;

In [ ]:
###

In [ ]:
WITH silver AS (
SELECT year, month,
    RANK() OVER w AS rank_silver
FROM sales
WHERE year = 2020 AND plan = 'silver'
WINDOW w AS (
    ORDER BY quantity DESC)
ORDER BY month ASC),

gold AS (
SELECT month,
    RANK() OVER w AS rank_gold
FROM sales
WHERE year = 2020 AND plan = 'gold'
WINDOW w AS (
    ORDER BY quantity DESC)
ORDER BY month ASC),

platinum AS (
SELECT month,
    RANK() OVER w AS rank_platinum
FROM sales
WHERE year = 2020 AND plan = 'platinum'
WINDOW w AS (
    ORDER BY quantity DESC)
ORDER BY month ASC)

SELECT silver.year,
    silver.month,
    silver.rank_silver AS silver,
    gold.rank_gold AS gold,
    platinum.rank_platinum AS platinum
FROM silver
    JOIN gold ON silver.month = gold.month
    JOIN platinum ON silver.month = platinum.month
ORDER BY month;

# **<font color='crimson'>Clustering</font>**

---

In [ ]:
###

In [ ]:
WITH temp_table AS (
SELECT
    user_id,
    adate,
    TO_DAYS(adate) - DENSE_RANK() OVER w AS group_id
FROM activity
WINDOW w AS (
    ORDER BY user_id, adate)
)

SELECT
    user_id,
    MIN(adate) AS day_start,
    MAX(adate) AS day_end,
    COUNT(*) AS day_count
FROM temp_table
GROUP BY user_id, group_id
ORDER BY user_id, day_start;

# **<font color='crimson'>Data cleaning</font>**

---

In [ ]:
###

In [ ]:
WITH rownumbers AS (
SELECT
    wdate, wtemp, precip,
    DENSE_RANK() OVER w AS rownum
FROM weather
WHERE wdate BETWEEN '2020-04-01' AND '2020-04-30'
WINDOW w AS (
    PARTITION BY precip
    ORDER BY precip DESC, wdate DESC)
)
SELECT
    wdate, wtemp,
    LAST_VALUE(precip) OVER w AS precip
FROM rownumbers
WHERE rownum = 1
WINDOW w AS (
    PARTITION BY precip
    ORDER BY precip DESC, wdate DESC
    rows between unbounded preceding and unbounded following)
ORDER BY wdate;

In [ ]:
###

In [ ]:
WITH avg_values AS (
SELECT
    wdate, precip,
    AVG(precip) OVER w AS avg_precip
FROM weather
WHERE (wdate BETWEEN '2020-03-06' AND '2020-03-11') \n
        OR (wdate BETWEEN '2020-06-01' AND '2020-06-06')
WINDOW w AS (
    ORDER BY wdate
    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
)

SELECT
    wdate, precip,
    coalesce(precip, avg_precip) AS fixed
FROM avg_values;

In [ ]:
###

In [ ]:
WITH bound AS (
SELECT
    TO_CHAR(wdate, 'YYYY-MM') AS month,
    percentile_cont(0.90) WITHIN GROUP (ORDER BY pressure) AS perc_90
FROM weather
GROUP BY TO_CHAR(wdate, 'YYYY-MM')
)

SELECT
    TO_CHAR(wdate, 'YYYY-MM') AS month,
    COUNT(pressure) FILTER (WHERE pressure > bound.perc_90) AS upper_values
FROM weather
    JOIN bound ON TO_CHAR(wdate, 'YYYY-MM') = bound.month
GROUP BY TO_CHAR(wdate, 'YYYY-MM')
ORDER BY month;